# MLP with TensorFlow Keras

In [1]:
import tensorflow as tf # type: ignore
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import json


2024-04-22 09:03:17.683020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [52]:
data = pd.read_csv("../data/iris.csv")

In [53]:
df_x = data.iloc[:, 1:5]
df_y = data.iloc[:, 5]

In [61]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Encode target labels
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df_y)

# Convert integers to one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# Split the data into train and test sets
x_train, x_test, y_train, y_test = train_test_split(df_x, onehot_encoded, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(x_train)
X_test_scaled = scaler.transform(x_test)

# Train MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp_classifier.fit(X_train_scaled, y_train)

# Predict on test set
y_pred = mlp_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0


In [30]:
with open('../models/relu_b.json', 'r') as f:
    json_data = json.load(f)

# Extract data from JSON
input_data = np.array(json_data['case']['input'])
target_data = np.array(json_data['case']['target'])
initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]

# Define the model architecture
model = tf.keras.Sequential()
for i, layer in enumerate(json_data['case']['model']['layers']):
    if i == 0:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], use_bias=True, activation=layer['activation_function'], input_shape=(json_data['case']['model']['input_size'],), kernel_initializer=tf.constant_initializer(initial_weights[i][1:]), bias_initializer=tf.constant_initializer(initial_weights[i][0])))
    else:
        model.add(tf.keras.layers.Dense(layer['number_of_neurons'], activation=layer['activation_function'], kernel_initializer=tf.constant_initializer(initial_weights[i][1:]), bias_initializer=tf.constant_initializer(initial_weights[i][0])))

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=json_data['case']['learning_parameters']['learning_rate']),
              loss='mean_squared_error', metrics=['accuracy'])

# Train the model
for epoch in range(json_data['case']['learning_parameters']['max_iteration']):
    history = model.fit(input_data, target_data, epochs=1, batch_size=json_data['case']['learning_parameters']['batch_size'], verbose=0)
    error = history.history['loss'][0]
    if error < json_data['case']['learning_parameters']['error_threshold']:
        print(f"Training stopped at epoch {epoch+1} because error threshold reached.")
        break

# Evaluate the model
final_weights = [layer.get_weights() for layer in model.layers]

# Check if final weights match the expected values
# Print final weights obtained from the model
print("Final weights:")
for i, (weights, biases) in enumerate(final_weights):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(weights)
    print("Biases:")
    print(biases)

# Print expected final weights from the JSON data
print("\nExpected final weights:")
for i, layer_weights in enumerate(json_data['expect']['final_weights']):
    print(f"Layer {i + 1}:")
    print("Weights:")
    print(layer_weights[1:])
    print("Biases: ")
    print(layer_weights[0])

Final weights:
Layer 1:
Weights:
[[ 0.30110002  0.5095      0.4335    ]
 [-0.49633333 -0.9683333   0.43033332]]
Biases:
[-0.20366667  0.16833334  0.96166664]

Expected final weights:
Layer 1:
Weights:
[[0.3033, 0.5285, 0.3005], [-0.489, -0.905, 0.291]]
Biases: 
[-0.211, 0.105, 0.885]


In [117]:
with open('../models/linear.json', 'r') as f:
    json_data = json.load(f)

# Extract data from JSON
input_size = json_data['case']['model']['input_size']
input_data = np.array(json_data['case']['input'])
target_data = np.array(json_data['case']['target'])
initial_weights = [np.array(layer) for layer in json_data['case']['initial_weights']]

# Define the model architecture
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(json_data['case']['model']['input_size'],)))

layers_json = json_data['case']['model']['layers']
for i, layer_json in enumerate(layers_json):
    layer = tf.keras.layers.Dense(units=layer_json['number_of_neurons'], activation=layer_json['activation_function'])

    if (i == 0):
        layer.build(input_shape=(input_size,))
    else:
        layer.build(input_shape=(layers_json[i-1]['number_of_neurons'],))

    weights = np.array(initial_weights[i][1:])
    bias = initial_weights[i][0]
    layer.set_weights([weights, bias])
    model.add(layer)

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
              loss='mean_squared_error')

model.fit(input_data, target_data, epochs=1, batch_size=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - loss: 0.2167


In [118]:
# model.summary()
model.get_weights()

[array([[ 0.52      ,  0.24000001, -0.8       ],
        [ 0.14      , -0.7866667 ,  0.46666667]], dtype=float32),
 array([0.14      , 0.31333333, 0.16666666], dtype=float32)]